In [ ]:
import os
hostname = os.popen("hostname").read().split("\n")[0]
if(hostname != "reckoner1429-Predator-PH315-52"):
    from google.colab import drive
    from google.colab import drive
    drive.mount('/content/gdrive')

    ! chmod 755 "/content/gdrive/My Drive/collab-var.sh"
    ! "/content/gdrive/My Drive/collab-var.sh"
    %cd "/content/gdrive/My Drive/github/video-emotion-recognition"
    from utils.hyparam_config import HyperparamConfigA4 as HyParamCfg
else:
    from utils.hyparam_config import HyperparamConfigTest as HyParamCfg

In [ ]:
import librosa
import librosa.display
import numpy as np
import utils.data_util as data_util
import utils.preprocess_util as preprocess_util
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Concatenate
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import time
import utils.local_config as local_config

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        print(gpu)
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
  print("no gpu available")

# Load Data

## Load file names

In [ ]:
if(local_config.DATASET_NAME == 'SAVEE'):
    dataset = preprocess_util.SAVEE()
elif(local_config.DATASET_NAME == 'RAVDESS'):
    dataset = preprocess_util.RAVDESS()
SEED = 0 
X_train_audio, X_test_audio, Y_train_audio, Y_test_audio = dataset.load_audio_filenames(SEED, 0.2)
X_train_face, X_test_face, Y_train_face, Y_test_face = dataset.load_visual_filenames(SEED, 0.2)
print(X_train_audio)
print(Y_train_face)

## Define some parameters

In [ ]:
INPUT_WIDTH = 224
INPUT_HEIGHT = 224
BATCH_SIZE = HyParamCfg['batch_size']
N_CLASSES = len(dataset.emotion_classes)
epochs = HyParamCfg['epochs']
ITERATION = HyParamCfg['iteration']

## Create the generators using filenames and labels

In [ ]:
X_train_face_gen = data_util.FaceDataGenerator(X_train_face, Y_train_face, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)
X_val_face_gen = data_util.FaceDataGenerator(X_test_face, Y_test_face, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)

In [ ]:
X_train_audio_gen = data_util.AudioDataGenerator(X_train_audio, Y_train_audio, BATCH_SIZE, INPUT_WIDTH, INPUT_WIDTH)
X_val_audio_gen = data_util.AudioDataGenerator(X_test_audio, Y_test_audio, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)

In [ ]:
X_train_gen = data_util.MultimodalDataGenerator(X_train_face, X_train_audio, Y_train_face, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)
X_val_gen = data_util.MultimodalDataGenerator(X_test_face, X_test_audio, Y_test_face, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)

# Define Functions for Training

## Define function to pretrain the model

In [ ]:
def fine_tune(model, train_data_gen, val_data_gen, epochs, network):
    output_layer = Dense(N_CLASSES, activation = 'softmax')(model.layers[-2].output)
    model = Model(model.input, output_layer, name = 'ftm-' + model.name + '-' + network + '-' + str(BATCH_SIZE))
    model.compile(optimizer = HyParamCfg['optimizer_ft'], loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    
    model_save_dir_path = local_config.MODEL_SAVE_DIR + '/' + 'ftm-' + str(SEED)+ '/' + 'saved_models' 
    model_save_path = model_save_dir_path + '/' + model.name + '.h5'   

    if(not os.path.exists(model_save_path)):
        history = model.fit(train_data_gen, validation_data = val_data_gen, epochs = epochs, verbose = 2)

        plt.plot(history.history['categorical_accuracy'])

        if(not os.path.exists(model_save_dir_path)):
            os.makedirs(model_save_dir_path)
        model.save(model_save_path)

        model_history_path = local_config.MODEL_SAVE_DIR + '/' + 'ftm-' + str(SEED)+ '/' + 'history'
        if(not os.path.exists(model_history_path)):
            os.makedirs(model_history_path)
        np.save(model_history_path + '/' + model.name + '-history.npy',history.history)
    
        model.evaluate(val_data_gen)

## Define the function for tranining fusion network

In [ ]:
def train_fusion_network(ptrn_model_name, ptrn_face_batch_size, ptrn_audio_batch_size, X_train_gen, X_val_gen, epochs):
    model_face = keras.models.load_model(os.path.join(local_config.MODEL_SAVE_DIR, 'ftm-' + str(SEED), 'saved_models/ftm-' + ptrn_model_name + '-face-' + str(ptrn_face_batch_size) + '.h5'))
    for layer in model_face.layers:
        layer._name = layer.name + '-face'
        layer.trainable = False
        output_layer_face = (model_face.layers[-2].output)
    
    model_audio = keras.models.load_model(os.path.join(local_config.MODEL_SAVE_DIR, 'ftm-' + str(SEED), 'saved_models/ftm-' + ptrn_model_name + '-audio-' + str(ptrn_face_batch_size) + '.h5'))
    for layer in model_audio.layers:
        layer._name = layer.name + '-audio'
        layer.trainable = False
    output_layer_audio = (model_audio.layers[-2].output)

    # Construct the fusion network
    layer = Concatenate()([output_layer_face, output_layer_audio])

    layer = Dense(2048, activation=HyParamCfg['layer1_activation'], kernel_initializer = HyParamCfg['layer1_kernel_intializer'], kernel_regularizer = HyParamCfg['layer1_kernel_regularizer'], activity_regularizer = HyParamCfg['layer1_activity_regulazier'])(layer)

    layer = HyParamCfg['layer1_dropout'](layer)

    layer = Dense(N_CLASSES, activation = 'softmax')(layer)

    model = Model([model_face.input, model_audio.input], layer, name = ptrn_model_name + '-' + str(ptrn_face_batch_size) + '-' + str(ptrn_audio_batch_size) + '-' + str(BATCH_SIZE) + '-' + str(ITERATION))

    # Compile the fusion network model
    model.compile(optimizer = HyParamCfg['optimizer_fusion'], loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

    history = model.fit(X_train_gen, validation_data = X_val_gen, epochs=epochs, verbose = 2)

    model_save_dir_path = os.path.join(local_config.MODEL_SAVE_DIR, 'iteration-' + str(ITERATION), 'saved_models')
    if(not os.path.exists(model_save_dir_path)):
        os.makedirs(model_save_dir_path)
    model.save(os.path.join(model_save_dir_path, model.name + '.h5'))

    model_history_path = os.path.join(local_config.MODEL_SAVE_DIR, 'iteration-' + str(ITERATION), 'history')
    if(not os.path.exists(model_history_path)):
        os.makedirs(model_history_path)
    np.save(os.path.join(model_history_path, model.name + '-history.npy'),history.history)

    plt.plot(history.history['loss'])
    plt.plot(history.history['categorical_accuracy'])

    model.evaluate(X_val_gen)

# Train the Networks

### Train the fusion Network

In [ ]:
ptrn_face_batch_size = BATCH_SIZE
ptrn_audio_batch_size = BATCH_SIZE

def train(base_model):
  print('training using ' + base_model.name + ', iteration = ' + HyParamCfg['iteration'])
  fine_tune(base_model, X_train_face_gen, X_val_face_gen, epochs, 'face')
  print("face fine tuned")
  fine_tune(base_model, X_train_audio_gen, X_val_audio_gen, epochs, 'audio')
  print("audio fine tuned")
  train_fusion_network(base_model.name, ptrn_face_batch_size, ptrn_audio_batch_size, X_train_gen, X_val_gen, epochs)

In [ ]:
train(tf.keras.applications.Xception())